# Логистическая регрессия. Практическая работа

## Цель практической работы

Вы уже делали задание, в котором с помощью метода ближайших соседей оценивали склонность клиента банка откликнуться или не откликнуться на предложение.

В этом практическом задании ваши цели:
*  решить эту же задачу с помощью логистической регрессии;
*  потренироваться в подборе порога; 
*  потренироваться в подборе гиперпараметров модели.

## Что входит в работу

*  Загрузить данные для задачи.
*  Обучить метод ближайших соседей с заданным количеством соседей k, вычислить метрики.
*  Обучить логистическую регрессию с параметрами по умолчанию, вычислить метрики.
*  Подобрать порог модели, вычислить метрики.
*  Подобрать гиперпараметр С (константа регуляризации) модели, вычислить метрики.

## Что оценивается

*  Выполнены все этапы задания: код запускается, отрабатывает без ошибок; подробно и обоснованно написаны текстовые выводы, где это требуется.

## Формат сдачи
Выполните предложенные задания — впишите свой код (или, если требуется, текст) в ячейки после комментариев. 

*Комментарии — это текст, который начинается с символа #. Например: # ваш код здесь.*

Сохраните изменения, используя опцию Save and Checkpoint из вкладки меню File или кнопку Save and Checkpoint на панели инструментов. Итоговый файл в формате .ipynb (файл Jupyter Notebook) загрузите в личный кабинет и отправьте на проверку.

In [1]:
# подключим библиотеки
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
# считаем данные
data = pd.read_csv('ClientsData.csv')

In [3]:
data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


В этом ноутбуке нам придётся подбирать гиперпараметры модели, а ещё порог. Поэтому, чтобы не переобучиться, разобъём данные на трейн, валидацию и тест.

*  Обучать модели будем на тренировочных данных.
*  Подбирать необходимые величины — по валидации.
*  Оценивать качество — на тесте.

In [81]:
# разделим данные на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

В задании по методу ближайших соседей было найдено, что оптимальное число соседей k = 9.

Обучите на тренировочных данных KNN с k = 9 и выведите матрицу ошибок, а также значение метрик precision и recall на тестовых данных.

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

In [24]:
# ваш код здесь
kn = KNeighborsClassifier(n_neighbors=9)
kn.fit(Xtrain, ytrain)
confusion_matrix(kn.predict(Xtest), ytest)

array([[3982,  553],
       [  26,    6]], dtype=int64)

In [25]:
print('точность:', precision_score(kn.predict(Xtest), ytest))
print('полнота:', recall_score(kn.predict(Xtest), ytest))

точность: 0.01073345259391771
полнота: 0.1875


Какой вывод можно сделать:
- для класса 0 — клиент не откликнулся — мы получили достаточно высокие значения TP в том числе потому, что представителей этого класса больше;
- для класса 1 — клиент откликнулся — мы получили низкие значения TN.

Поэтому значения precision и recall низкие. Модель даёт неудовлетворительные результаты, так как находит мало клиентов, которые откликнутся на предложение.



Обучите логистическую регрессию с параметрами по умолчанию и посмотрите на метрики.

Везде дальше при оценке метрик надо выводить confusion_matrix, precision и recall.

In [31]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(Xtrain, ytrain)
confusion_matrix(model.predict(Xtest), ytest)
# ваш код здесь

array([[4004,  559],
       [   4,    0]], dtype=int64)

In [32]:
print('точность:', precision_score(model.predict(Xtest), ytest))
print('полнота:', recall_score(model.predict(Xtest), ytest))

точность: 0.0
полнота: 0.0


Наша цель — найти как можно больше клиентов, которые откликнутся на предложение. А модель таких не находит. 

Мы помним, что метод predict_proba у логистической регрессии предсказывает математические (то есть корректные) вероятности классов. Предскажите вероятности классов с помощью обученной логистической регрессии на тестовых данных и выведите вероятности положительного класса для первых десяти объектов. 

Глядя на полученные вероятности, попробуйте объяснить, почему вы получили именно такую матрицу ошибок и такие значения точности с полноты.

In [37]:
# ваш код здесь
model.predict_proba(Xtest[:10])

array([[0.92867082, 0.07132918],
       [0.88469094, 0.11530906],
       [0.70811808, 0.29188192],
       [0.78090239, 0.21909761],
       [0.86126565, 0.13873435],
       [0.95507473, 0.04492527],
       [0.93575353, 0.06424647],
       [0.96411357, 0.03588643],
       [0.94284198, 0.05715802],
       [0.93128801, 0.06871199]])

In [40]:
#Ваше объяснение здесь

# Левый столбец указывает вероятность принадлежности к классу 0 (не откликнулись), а правый к классу 1 (откликнулись).
# По умолчанию - порог перевода >= 0.5 определяет принадлежность к классу. Его можно изменять. В данных случаях - все объекты 
# определились в 0 класс.

Давайте уточним цель. Пусть нам нужно найти как можно больше клиентов, которые откликнутся на предложение, то есть максимизировать полноту (recall). 

При этом хочется, чтобы точность модели (precision) не была очень низкой. Заказчик посмотрел на результаты работы KNN и потребовал, чтобы точность была не ниже, чем у KNN: $precision \geq 0.13$.

Давайте будем изменять порог для перевода вероятности в классы так, чтобы:
*   максимизировать значение recall
*   при условии, что $precision \geq 0.13$.

Если мы будем подбирать порог по тестовой выборке, то, по сути, обучимся на ней и, значит, переобучимся. Это плохо. 

Поэтому предскажите вероятности на валидационной выборке и подберите порог по ней (Xval, yval), а затем посмотрите, какое качество для найденного порога вы получите на тестовых данных.

In [186]:
probs_val = kn.predict_proba(Xval)[:, 1]# ваш код здесь

max_recall = -1
thr = -1
prec = -1
classes = []

for threshold in np.arange(0.05, 1, 0.001):
    classes.append((probs_val > threshold).astype(int))
for i in range(950):
    if max_recall < recall_score(yval, classes[i]) and precision_score(yval, classes[i]) >= 0.13:
        max_recall = recall_score(yval, classes[i])
        prec = precision_score(yval, classes[i])
        thr = threshold


print(f'максимальная полнота: {max_recall},\nточность: {prec}, \nпорог: {thr}')
    # для каждого значения порога переведbите вероятности в классы
    # посчитайте метрики

# напечатайте порог, для которого получается максимальная полнота, при precision >= 0.13

максимальная полнота: 0.36827195467422097,
точность: 0.1388888888888889, 
порог: 0.9990000000000009


In [190]:
# по найденному порогу переведите вероятности в классы на тесте и напечатайте метрики
class_test = (kn.predict_proba(Xtest)[:, 1] > threshold).astype(int)
print(f'Полнота: {recall_score(ytest, class_test)},\nТочность: {precision_score(ytest, class_test)}, \nПорог: {thr}')

Полнота: 0.0,
Точность: 0.0, 
Порог: 0.9990000000000009


In [195]:
confusion_matrix(ytest,class_test)

array([[4008,    0],
       [ 559,    0]], dtype=int64)

Сделайте вывод. Смогли ли мы с помощью подбора порога добиться большего значения recall, чем у KNN? 

In [193]:
# ваш вывод здесь
# При таком большом пороге произошло недообучение. Просто, большинство объектов упало нужным образов в выборке
# Xval, yval. И с условием в точности >= 13%, Порог вышел слишком большим.

А ещё, чтобы улучшить качество предсказания, можно подбирать значение гиперпараметра C у логистической регрессии. Для каждого значения C придётся подбирать свой порог, поэтому:  

1. Обучите для значений C из диапазона [0.05, 0.15, 0.25, ...., 10.05] логистическую регрессию (на тренировочных данных).

2. Для каждой из обученных моделей во внутреннем цикле подберите оптимальный порог (как в предыдущем задании) — на валидационных данных.



В качестве результата выведите значение C и порога для модели, которая даёт наилучшие значения метрик (наибольший recall при ограничении на $precision \geq 0.13$).

Также напечатайте полученные метрики (матрицу ошибок, точность и полноту) для лучшей модели — на тестовых данных.

In [201]:
regs = []
recalls = []
thresholds = []
precisions = []

for reg in np.arange(0.001, 1, 0.01):
    model1 = LogisticRegression(C=reg, random_state=42)
    model1.fit(Xtrain, ytrain)

    # обучите логистическую регрессию с C=reg

    probs_val = model1.predict_proba(Xval)[:, 1]
    max_recall = -1
    thr = -1
    prec = -1
    reg_C = -1
    classes = []

    for threshold in np.arange(0.05, 1, 0.001):
        classes.append((probs_val > threshold).astype(int))
    for i in range(950):
        if max_recall < recall_score(yval, classes[i]) and precision_score(yval, classes[i]) >= 0.13:
            max_recall = recall_score(yval, classes[i])
            prec = precision_score(yval, classes[i])
            thr = threshold
            reg_C = reg
        # подберите оптимальный порог как в задании выше

    recalls.append(max_recall)
    thresholds.append(thr)
    precisions.append(prec)
    regs.append(reg_C)


print(f'Максимальная полнота: {max_recall},\nТочность: {prec}, \nПорог: {thr}, \nC:{reg_C}')

Максимальная полнота: 0.9093484419263456,
Точность: 0.13001215066828675, 
Порог: 0.9990000000000009, 
C:0.9909999999999999


In [203]:
model2 = LogisticRegression(C = 0.9909999999999999, random_state=42)
model2.fit(Xtest, ytest)
threshold = 0.9990000000000009

probs_val2 = model2.predict_proba(Xtest)[:, 1]
model_predict = (probs_val2 > threshold).astype(int)
confusion_matrix(ytest, model_predict)

array([[4008,    0],
       [ 559,    0]], dtype=int64)

In [206]:
print(f'Максимальная полнота: {recall_score(ytest, model2.predict(Xtest))},\nТочность: {precision_score(ytest, model2.predict(Xtest))}, \nПорог: {threshold}, \nC: 0.9909999999999999')

Максимальная полнота: 0.0017889087656529517,
Точность: 0.3333333333333333, 
Порог: 0.9990000000000009, 
C: 0.9909999999999999


In [ ]:
# выведите значения C, precision, recall, threshold для наилучшей по заданным условиям модели

Влияет ли изменение гиперпараметра C на качество модели (и, соответственно, метрики) в этой задаче?

 ВАШ ВЫВОД ЗДЕСЬ


Можно сделать следующиий вывод: гиперпараметр С и метрики явно сильно влияют на качество модели, но по разному.
При небольшом объеме данных мы можем просто вывести все в TP и при этом получить хорошие метрики, практически при любых
значениях параметров. Но при установке таких же параметров на большие данные - они не покажут хорошего результата.

Вывод: не во всех выборках изменение параметров может помочь повлиять на качество модели. Иногда просто может не хватать данных,
чтобы модель могла верна определять класс объекта.

Ответьте развёрнуто на следующие вопросы:

* Удалось ли при помощи логистической регрессии и подбора порога превзойти качество метода ближайших соседей в этой задаче? 

* Смогли ли мы при помощи этой модели получить высокий recall при ограничениях, поставленных заказчиком?


In [208]:
# ваш вывод здесь